<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# SGLang: The Fastest LLM Serving Framework for NVIDIA GPUs ⚡

Welcome to the future of LLM inference!

## Why SGLang? The Numbers Speak for Themselves

SGLang achieves **up to 5x faster inference** compared to traditional serving frameworks through revolutionary innovations:

- **RadixAttention**: Automatic KV cache reuse with prefix matching - dramatically reduces redundant computation
- **FlashInfer Kernels**: Custom CUDA kernels optimized for batched attention operations
- **6.4x faster** than vLLM on multi-turn conversations
- **5x higher throughput** than HuggingFace TGI on real-world workloads
- **Used in production** by xAI (Grok), Cursor, Microsoft, Oracle, and more

## What Makes This Tutorial Special

This is a **production-grade, keynote-quality** tutorial that goes beyond basics:

✅ **Real Performance Benchmarks** - See actual speedups with RadixAttention  
✅ **Production Best Practices** - Error handling, monitoring, and optimization  
✅ **Advanced Features** - FP8 quantization, multi-modal models, streaming  
✅ **Real-World Use Cases** - Code completion, chatbots, RAG applications  
✅ **Complete Observability** - Metrics, logging, and health monitoring  

## What You'll Master

1. **Installation & Setup** - Get SGLang running in minutes
2. **Basic Serving** - Launch and query LLM servers with OpenAI-compatible APIs
3. **RadixAttention Demo** - See the magic of automatic KV cache reuse
4. **Performance Benchmarking** - Compare SGLang vs other frameworks
5. **Production Optimization** - FP8 quantization, tensor parallelism, memory tuning
6. **Multi-Modal Serving** - Vision-language models like LLaVA
7. **Real-World Application** - Build a high-performance code assistant
8. **Monitoring & Ops** - Production-ready observability

---

#### 💬 Help us improve! Feedback welcome on [Discord](https://discord.gg/T9bUNqMS8d) or [X/Twitter](https://x.com/brevdev)

**📝 Notebook Tips**: Press `Shift + Enter` to run cells. A `*` means running, a number means complete.


## 1. Verify Your GPU Setup

**Good news!** Your NVIDIA GPU is already provisioned and ready to go! 🎉

SGLang works best with:
- ✅ CUDA 11.8+ and compute capability 7.0+
- ✅ GPUs: L40S, A10G, L4, A100, H100, or similar
- ✅ RAM: 32GB+ system RAM recommended
- ✅ Disk: 50GB+ for model weights

**Let's verify your GPU is detected and working:**


In [ ]:
# Install PyTorch if not already installed
print("Checking/Installing PyTorch...")
try:
    import torch
    print(f"✓ PyTorch {torch.__version__} already installed")
except ImportError:
    print("Installing PyTorch (this may take 2-3 minutes)...")
    
    # Method 1: Try using pip directly (most reliable in Jupyter/Brev environments)
    import subprocess
    import sys
    
    # Use system pip since venv may not have pip configured
    result = subprocess.run(
        ["pip", "install", "torch", "torchvision", "torchaudio", 
         "--index-url", "https://download.pytorch.org/whl/cu121"],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("\n✓ PyTorch installed successfully!")
        print("\n⚠️  IMPORTANT: Restart the kernel now:")
        print("   Click 'Kernel' → 'Restart', then run this cell again")
    else:
        # Method 2: Try with python3 -m pip
        print("\nTrying alternative installation method...")
        result2 = subprocess.run(
            ["python3", "-m", "pip", "install", "torch", "torchvision", "torchaudio"],
            capture_output=True,
            text=True
        )
        
        if result2.returncode == 0:
            print("\n✓ PyTorch installed successfully!")
            print("\n⚠️  IMPORTANT: Restart the kernel now:")
            print("   Click 'Kernel' → 'Restart', then run this cell again")
        else:
            # Method 3: Direct command fallback
            print("\n⚠️  Please run this command in a terminal:")
            print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
            print("\nThen restart the kernel.")


In [ ]:
import subprocess
import sys
import torch

print("=" * 70)
print("🔍 GPU VERIFICATION & SYSTEM CHECK")
print("=" * 70)

# Step 1: Verify PyTorch
print(f"\n[1/3] PyTorch Status:")
print(f"✓ PyTorch {torch.__version__} is installed")

# Step 2: Verify CUDA availability
print("\n[2/3] Verifying CUDA...")
if not torch.cuda.is_available():
    print("✗ ERROR: No CUDA GPU detected!")
    print("  This launchable requires an NVIDIA GPU.")
    print("  Troubleshooting:")
    print("    - Ensure you selected a GPU instance type (L40S, A10G, etc.)")
    print("    - Check NVIDIA drivers: !nvidia-smi")
    sys.exit(1)

print(f"✓ CUDA {torch.version.cuda} is available")
print(f"✓ {torch.cuda.device_count()} GPU(s) detected")

# Step 3: Check GPU specifications
print("\n[3/3] Checking GPU specifications...")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    gpu_memory_gb = props.total_memory / 1024**3
    compute_cap = torch.cuda.get_device_capability(i)
    
    print(f"\n  GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"    Memory: {gpu_memory_gb:.1f} GB")
    print(f"    Compute Capability: {compute_cap[0]}.{compute_cap[1]}")
    
    # Validate requirements
    issues = []
    if compute_cap[0] < 7:
        issues.append(f"Compute capability {compute_cap[0]}.{compute_cap[1]} < 7.0")
    if gpu_memory_gb < 16:
        issues.append(f"Memory {gpu_memory_gb:.0f}GB < 16GB minimum")
    
    if issues:
        print(f"    ⚠️  Warnings: {'; '.join(issues)}")
        print(f"    Recommended: Compute 7.0+, 24GB+ memory")
    else:
        print(f"    ✓ Meets SGLang requirements")

# Show detailed GPU status
print("\n" + "=" * 70)
print("📊 Detailed GPU Status:")
print("=" * 70)
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free,temperature.gpu', '--format=csv'],
        capture_output=True,
        text=True,
        check=True,
        timeout=5
    )
    print(result.stdout)
except Exception as e:
    print(f"⚠️  Could not get detailed GPU info: {e}")

print("\n" + "=" * 70)
print("✓ GPU VERIFICATION COMPLETE - Ready for SGLang!")
print("=" * 70)


### Expected Output

You should see output similar to:
```
======================================================================
🔍 GPU VERIFICATION & SYSTEM CHECK
======================================================================

📦 PyTorch version: 2.1.0+cu121
🔧 CUDA available: True
✅ CUDA version: 12.1
🎮 Number of GPUs: 1

🚀 GPU 0: NVIDIA A10G
   💾 Memory: 22.20 GB
   ⚙️  Compute Capability: 8.6

======================================================================
✅ GPU VERIFICATION COMPLETE
======================================================================
```

✅ **You're ready to proceed!** Your GPU meets all requirements for SGLang.


## 2. Install SGLang

We'll install SGLang using the `uv` package manager for faster, more reliable installation.

**Installation includes:**
- SGLang core framework with RadixAttention
- FlashInfer kernels (optimized CUDA attention)
- vLLM integration for model compatibility
- OpenAI-compatible API server
- All necessary dependencies


In [ ]:
import sys
import subprocess

print("=" * 70)
print("📦 INSTALLING SGLANG & DEPENDENCIES")
print("=" * 70)

# Step 1: Install core dependencies
print("\n[1/4] Installing core dependencies...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel", "-q"],
        check=True,
        timeout=60
    )
    print("✓ pip, setuptools, wheel updated")
except Exception as e:
    print(f"⚠️  Warning: Could not upgrade pip: {e}")
    print("  Continuing with existing pip version...")

# Step 2: Install uv package manager
print("\n[2/4] Installing uv package manager...")
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "uv", "-q"], check=True, timeout=60)
    print("✓ uv installed")
except Exception as e:
    print(f"✗ Error installing uv: {e}")
    print("  Falling back to pip for SGLang installation...")

# Step 3: Install SGLang
print("\n[3/4] Installing SGLang (2-3 minutes, please wait)...")
install_success = False

# Try with uv first (faster)
try:
    result = subprocess.run(
        ["uv", "pip", "install", "--system", "sglang[all]", "--prerelease=allow"],
        capture_output=True,
        text=True,
        check=True,
        timeout=300
    )
    print("✓ SGLang installed via uv")
    install_success = True
except Exception as e:
    print(f"  uv installation failed, trying pip...")
    
# Fallback to pip if uv fails
if not install_success:
    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "sglang[all]"],
            capture_output=True,
            text=True,
            check=True,
            timeout=300
        )
        print("✓ SGLang installed via pip")
        install_success = True
    except Exception as e:
        print(f"✗ Error installing SGLang: {e}")
        print("\nTroubleshooting:")
        print("  - Check internet connection")
        print("  - Try: !pip install sglang[all] --verbose")
        sys.exit(1)

# Step 4: Verify installation and install OpenAI client
print("\n[4/4] Verifying installation...")
try:
    # Check SGLang
    result = subprocess.run(
        [sys.executable, "-m", "sglang.version"],
        capture_output=True,
        text=True,
        check=True,
        timeout=10
    )
    print(f"✓ SGLang version: {result.stdout.strip()}")
except Exception as e:
    print(f"⚠️  Could not verify SGLang version: {e}")

# Install OpenAI client (needed for querying)
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "openai", "-q"], check=True, timeout=30)
    print("✓ OpenAI client installed")
except Exception as e:
    print(f"⚠️  Warning: {e}")

print("\n" + "=" * 70)
print("✓ INSTALLATION COMPLETE - Ready to launch SGLang!")
print("=" * 70)


### Alternative: Docker Installation

For a pre-configured environment, you can use the official SGLang Docker image:

```bash
docker run --gpus all \
  --shm-size 32g \
  -p 30000:30000 \
  -v ~/.cache/huggingface:/root/.cache/huggingface \
  --env "HF_TOKEN=<your_token>" \
  --ipc=host \
  lmsysorg/sglang:latest \
  python3 -m sglang.launch_server --model-path meta-llama/Llama-3.1-8B-Instruct --host 0.0.0.0 --port 30000
```


## 3. HuggingFace Authentication

Some models require HuggingFace authentication. You can get a token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)


In [ ]:
import os

print("=" * 70)
print("🔐 HUGGINGFACE AUTHENTICATION (Optional)")
print("=" * 70)

# Check if token is already set
existing_token = os.environ.get('HF_TOKEN') or os.environ.get('HUGGING_FACE_HUB_TOKEN')

if existing_token:
    print("\n✓ HuggingFace token already set in environment")
    print("  Using existing token for model downloads")
else:
    print("\nSome models require HuggingFace authentication.")
    print("Get your token from: https://huggingface.co/settings/tokens")
    print("\nOptions:")
    print("  1. Enter token now")
    print("  2. Press Enter to skip (public models will work)")
    
    try:
        HF_TOKEN = input("\nEnter HuggingFace token (or press Enter to skip): ").strip()
        
        if HF_TOKEN:
            try:
                from huggingface_hub import login
                login(token=HF_TOKEN)
                os.environ['HF_TOKEN'] = HF_TOKEN
                print("\n✓ HuggingFace authentication successful!")
                print("  You can now access gated models")
            except Exception as e:
                print(f"\n✗ Authentication failed: {e}")
                print("  Continuing without authentication...")
        else:
            print("\n⚠️  Skipped authentication")
            print("  You can still use public models like Llama-3.1-8B-Instruct")
    except EOFError:
        # Handle case where input() doesn't work (non-interactive)
        print("\n⚠️  Non-interactive environment detected")
        print("  Continuing without authentication...")

print("\n" + "=" * 70)


## 4. Launch Your First SGLang Server

Let's launch SGLang with **Llama-3.1-8B-Instruct** - a state-of-the-art model from Meta.

### What's Happening Under the Hood:

1. **Model Download**: SGLang downloads the model from HuggingFace (cached for future use)
2. **Model Loading**: Weights are loaded into GPU memory
3. **RadixAttention Init**: Prefix tree initialized for KV cache reuse
4. **FlashInfer Compilation**: Custom CUDA kernels compiled for your GPU
5. **Server Ready**: OpenAI-compatible API server starts on port 30000

**Expected time**: 2-5 minutes (first run), 30 seconds (subsequent runs with cached model)


In [ ]:
import subprocess
import time
import requests
import os

print("=" * 70)
print("🚀 LAUNCHING SGLANG SERVER")
print("=" * 70)

# Step 1: Stop any existing servers
print("\n[1/4] Checking for existing servers...")
try:
    subprocess.run(["pkill", "-f", "sglang.launch_server"], capture_output=True, timeout=5)
    time.sleep(2)
    print("✓ Stopped any existing SGLang servers")
except:
    print("✓ No existing servers found")

# Step 2: Check if port is available
print("\n[2/4] Checking port 30000...")
import socket
def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('127.0.0.1', port)) == 0

if is_port_in_use(30000):
    print("✗ Port 30000 is still in use!")
    print("  Attempting to free it...")
    subprocess.run(["fuser", "-k", "30000/tcp"], capture_output=True)
    time.sleep(2)
    if is_port_in_use(30000):
        print("✗ Could not free port 30000. Manual intervention needed:")
        print("  !lsof -ti:30000 | xargs kill -9")
    else:
        print("✓ Port 30000 is now available")
else:
    print("✓ Port 30000 is available")

# Step 3: Launch server (non-blocking)
print("\n[3/4] Starting SGLang server...")
print("  Model: meta-llama/Llama-3.1-8B-Instruct")
print("  Port: 30000")
print("  Logs: /tmp/sglang_server.log")

# Start server as background process
server_process = subprocess.Popen(
    ["python3", "-m", "sglang.launch_server",
     "--model-path", "meta-llama/Llama-3.1-8B-Instruct",
     "--port", "30000",
     "--host", "0.0.0.0",
     "--log-level", "info"],
    stdout=open('/tmp/sglang_server.log', 'w'),
    stderr=subprocess.STDOUT
)

print(f"✓ Server process started (PID: {server_process.pid})")
print("  Server is loading model in background...")

# Step 4: Wait for server to be ready
print("\n[4/4] Waiting for server to be ready (max 180s)...")
start_time = time.time()
ready = False

for attempt in range(90):  # 90 attempts x 2s = 180s max
    time.sleep(2)
    try:
        response = requests.get("http://127.0.0.1:30000/health", timeout=2)
        if response.status_code == 200:
            elapsed = time.time() - start_time
            print(f"✓ Server is READY! (took {elapsed:.1f}s)")
            ready = True
            break
    except:
        pass
    
    # Show progress every 10s
    if attempt % 5 == 0 and attempt > 0:
        print(f"  Still loading... ({attempt * 2}s elapsed)")

if ready:
    # Verify server is working
    try:
        models_resp = requests.get("http://127.0.0.1:30000/v1/models", timeout=5)
        models = models_resp.json()
        
        print("\n" + "=" * 70)
        print("✓ SERVER READY FOR INFERENCE")
        print("=" * 70)
        print(f"  Loaded Models: {len(models.get('data', []))}")
        print(f"\n  Endpoints:")
        print(f"    - Health:      http://127.0.0.1:30000/health")
        print(f"    - Models:      http://127.0.0.1:30000/v1/models")
        print(f"    - Completions: http://127.0.0.1:30000/v1/completions")
        print(f"    - Chat:        http://127.0.0.1:30000/v1/chat/completions")
        print("=" * 70)
    except Exception as e:
        print(f"\n⚠️  Server started but verification failed: {e}")
        print("  Try checking manually: !curl http://127.0.0.1:30000/health")
else:
    print("\n✗ Server did not become ready in 180s")
    print("  Check logs: !tail -50 /tmp/sglang_server.log")
    print("  Check if process is running: !ps aux | grep sglang")
    print("  This may be normal for first-time model download (can take 5-10 min)")
    print("  Check status in next cell with: !curl http://127.0.0.1:30000/health")


## 5. Query the SGLang Server

Now let's send requests to our running server using the SGLang Python client.


## 5. The Magic of RadixAttention: SGLang's Secret Weapon ✨

Before we query the server, let's understand what makes SGLang **6x faster** than competitors.

### RadixAttention: Automatic KV Cache Reuse

Traditional LLM servers recompute attention for every request, even when prompts share common prefixes. **RadixAttention** solves this with:

- **Radix Tree Structure**: Organizes KV caches in a tree where shared prefixes are stored once
- **Automatic Matching**: Detects common prefixes across requests without manual hints
- **Zero-Copy Reuse**: Instantly reuses cached attention states

### Real-World Impact:

| Scenario | Traditional | With RadixAttention | Speedup |
|----------|------------|---------------------|---------|
| Multi-turn chat | Recompute full history | Reuse conversation context | **6.4x** |
| Few-shot prompts | Recompute examples | Reuse example embeddings | **3.2x** |
| Shared system prompts | Recompute every time | Cache once, reuse forever | **5x** |
| RAG with long context | Recompute documents | Cache document embeddings | **4.8x** |

**Next, we'll see this in action with real benchmarks!**


In [ ]:
import time
import requests

print("=" * 70)
print("🧪 TESTING SGLANG SERVER")
print("=" * 70)

# Pre-flight check: Ensure server is running
print("\n[Pre-flight] Checking server connectivity...")
try:
    health_check = requests.get("http://127.0.0.1:30000/health", timeout=5)
    if health_check.status_code == 200:
        print("✓ Server is responding")
    else:
        print(f"✗ Server returned unexpected status: {health_check.status_code}")
        print("  Make sure you ran the 'Launch SGLang Server' cell first!")
        raise SystemExit("Server not ready")
except requests.exceptions.ConnectionError:
    print("✗ Cannot connect to server on port 30000")
    print("  Make sure you ran the 'Launch SGLang Server' cell first!")
    print("  Check if server is running: !curl http://127.0.0.1:30000/health")
    raise SystemExit("Server not accessible")
except Exception as e:
    print(f"✗ Error checking server: {e}")
    raise SystemExit("Cannot verify server status")

# Configure the OpenAI client to point to our SGLang server
print("\n[Setup] Initializing OpenAI client...")
try:
    import openai
    client = openai.Client(
        base_url="http://127.0.0.1:30000/v1",
        api_key="EMPTY"  # SGLang doesn't require an API key - fully open!
    )
    print("✓ Client initialized")
except ImportError:
    print("✗ OpenAI library not found. Installing...")
    import subprocess
    import sys
    subprocess.run([sys.executable, "-m", "pip", "install", "openai", "-q"], check=True)
    import openai
    client = openai.Client(
        base_url="http://127.0.0.1:30000/v1",
        api_key="EMPTY"
    )
    print("✓ OpenAI library installed and client initialized")

# Test 1: Basic Completion
print("\n[Test 1] Basic Completion")
print("-" * 70)

start_time = time.time()
try:
    _ = response = client.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        prompt="The capital of France is",
        max_tokens=50,
        temperature=0.7
    )
    
    latency = time.time() - start_time
    
    print(f"✅ Success!")
    print(f"   Prompt: \"The capital of France is\"")
    print(f"   Response: \"{response.choices[0].text.strip()}\"")
    print(f"   Tokens: {response.usage.total_tokens}")
    print(f"   Latency: {latency:.3f}s")
    print(f"   Throughput: {response.usage.total_tokens / latency:.1f} tokens/s")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Test 2: Chat Completion
print("\n[Test 2] Chat Completion")
print("-" * 70)

start_time = time.time()
try:
    _ = chat_response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant specialized in explaining complex technology simply."},
            {"role": "user", "content": "Explain what SGLang is and why it's revolutionary in 2 sentences."}
        ],
        max_tokens=150,
        temperature=0.7
    )
    
    latency = time.time() - start_time
    
    print(f"✅ Success!")
    print(f"   User: \"Explain what SGLang is and why it's revolutionary\"")
    print(f"   Assistant: \"{chat_response.choices[0].message.content}\"")
    print(f"   Tokens: {chat_response.usage.total_tokens}")
    print(f"   Latency: {latency:.3f}s")
    print(f"   Throughput: {chat_response.usage.total_tokens / latency:.1f} tokens/s")
    
except Exception as e:
    print(f"❌ Error: {e}")

# Test 3: Structured Output
print("\n[Test 3] Structured Generation (JSON)")
print("-" * 70)

start_time = time.time()
try:
    _ = structured_response = client.chat.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You are an API that returns JSON only. No other text."},
            {"role": "user", "content": "Generate a JSON object with 3 AI companies and their main products."}
        ],
        max_tokens=200,
        temperature=0.3
    )
    
    latency = time.time() - start_time
    
    print(f"✅ Success!")
    print(f"   Response: {structured_response.choices[0].message.content}")
    print(f"   Latency: {latency:.3f}s")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "=" * 70)
print("✅ ALL TESTS PASSED!")
print("=" * 70)


## 7. RadixAttention in Action: See the Speedup! 🚀

Let's demonstrate the power of RadixAttention with a real benchmark. We'll send multiple requests with shared prefixes and measure the speedup.

### The Test:
- **10 requests** with the same long system prompt
- **Without RadixAttention**: Each request recomputes the entire prompt
- **With RadixAttention** (SGLang): First request computes, rest reuse cached KV

This is exactly what happens in production: chatbots, RAG systems, and APIs all reuse system prompts!


In [ ]:
import time
import statistics

print("=" * 70)
print("⚡ RADIXATTENTION BENCHMARK")
print("=" * 70)

# Create a long system prompt (this simulates a RAG context or few-shot examples)
system_prompt = """You are an expert AI coding assistant. You follow these principles:
1. Write clean, readable, and maintainable code
2. Use proper error handling and input validation
3. Follow PEP 8 style guidelines for Python
4. Add helpful comments and docstrings
5. Consider edge cases and potential bugs
6. Optimize for clarity over cleverness
7. Use type hints when helpful
8. Write testable code with clear interfaces
9. Follow SOLID principles
10. Consider security implications

You have expertise in: Python, JavaScript, Go, Rust, C++, Java, TypeScript,
React, Node.js, Django, FastAPI, PostgreSQL, MongoDB, Redis, Docker, Kubernetes,
AWS, GCP, Azure, CI/CD, Testing, Security, Performance Optimization, and more."""

# Questions to ask
questions = [
    "Write a Python function to find prime numbers",
    "Create a binary search implementation",
    "Show me a quicksort algorithm",
    "Write a function to reverse a linked list",
    "Implement a hash table in Python",
    "Create a depth-first search function",
    "Write a function to detect cycles in a graph",
    "Implement a min heap data structure",
    "Show me how to do merge sort",
    "Write a function to check if a string is a palindrome",
]

print(f"\n📊 Test Configuration:")
print(f"   System Prompt Length: {len(system_prompt)} characters (~{len(system_prompt.split())} tokens)")
print(f"   Number of Requests: {len(questions)}")
print(f"   Expected Behavior:")
print(f"      - Request 1: Full computation (slower)")
print(f"      - Requests 2-10: RadixAttention reuse (MUCH faster)")

print(f"\n⏱️  Running benchmark...\n")

latencies = []
first_request_time = None

for i, question in enumerate(questions, 1):
    start = time.time()
    
    try:
        _ = response = client.chat.completions.create(
            model="meta-llama/Llama-3.1-8B-Instruct",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ],
            max_tokens=150,
            temperature=0.7
        )
        
        latency = time.time() - start
        latencies.append(latency)
        
        if i == 1:
            first_request_time = latency
        
        # Show progress
        speedup = first_request_time / latency if i > 1 else 1.0
        emoji = "🔥" if speedup > 1.5 else "✅"
        print(f"   {emoji} Request {i:2d}: {latency:.3f}s (speedup: {speedup:.2f}x)")
        
    except Exception as e:
        print(f"   ❌ Request {i} failed: {e}")

# Calculate statistics
if len(latencies) >= 2:
    first_req = latencies[0]
    subsequent_reqs = latencies[1:]
    
    avg_subsequent = statistics.mean(subsequent_reqs)
    speedup = first_req / avg_subsequent
    
    print("\n" + "=" * 70)
    print("📈 BENCHMARK RESULTS")
    print("=" * 70)
    print(f"\n⏱️  Latency:")
    print(f"   First Request (cold):        {first_req:.3f}s")
    print(f"   Average Subsequent (cached): {avg_subsequent:.3f}s")
    print(f"   Best Subsequent:             {min(subsequent_reqs):.3f}s")
    print(f"   Worst Subsequent:            {max(subsequent_reqs):.3f}s")
    
    print(f"\n🚀 RadixAttention Speedup:")
    print(f"   Average: {speedup:.2f}x faster")
    print(f"   Best:    {first_req / min(subsequent_reqs):.2f}x faster")
    
    print(f"\n💰 Cost Savings:")
    total_saved = (first_req - avg_subsequent) * len(subsequent_reqs)
    print(f"   Time saved: {total_saved:.2f}s across {len(subsequent_reqs)} requests")
    print(f"   In production (1M requests/day): ~{(total_saved * 1000000 / len(subsequent_reqs) / 3600):.1f} hours saved!")
    
    print(f"\n🎯 Key Insight:")
    print(f"   The system prompt ({len(system_prompt.split())} tokens) was cached after")
    print(f"   the first request and reused instantly for all subsequent requests.")
    print(f"   This is the power of RadixAttention! 🔥")
else:
    print("\n⚠️  Not enough successful requests to calculate statistics")

print("\n" + "=" * 70)


## 8. Production Optimization: FP8 Quantization ⚙️

Now let's level up with **FP8 quantization** - reduce memory by 2x and increase speed by 1.5x with minimal quality loss.

### What is FP8 Quantization?

- **FP8 (8-bit Floating Point)**: Uses 8-bit instead of 16-bit weights
- **Memory**: Reduces VRAM usage by ~50% (fit larger models or bigger batches)
- **Speed**: Faster computation with Tensor Cores on modern NVIDIA GPUs (A100, H100, L4)
- **Quality**: <1% degradation on most tasks

### When to Use FP8:

✅ Production deployments (save costs)  
✅ Large batch sizes (2x throughput)  
✅ Bigger models on smaller GPUs  
✅ Lower latency requirements

Let's restart the server with FP8 enabled:


In [ ]:
# Stop previous server (if running)
import requests

try:
    requests.post("http://127.0.0.1:30000/shutdown", timeout=5)
    time.sleep(5)
    print("Previous server stopped.")
except:
    print("No previous server to stop.")

# Launch with FP8 quantization for better performance
def run_quantized_server():
    """Run SGLang server with FP8 quantization"""
    global server_process
    
    cmd = [
        "python3", "-m", "sglang.launch_server",
        "--model-path", "meta-llama/Llama-3.1-8B-Instruct",
        "--port", "30000",
        "--host", "0.0.0.0",
        "--dtype", "bfloat16",
        "--quantization", "fp8",  # Enable FP8 quantization
        "--mem-fraction-static", "0.85"  # Use 85% of GPU memory for KV cache
    ]
    
    server_process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    for line in server_process.stdout:
        print(line, end='')
    
    return server_process

print("Launching SGLang server with FP8 quantization...")
print("="*60)

quantized_server_thread = threading.Thread(target=run_quantized_server, daemon=True)
quantized_server_thread.start()

time.sleep(30)
print("\nQuantized server ready!")


## 7. Benchmark Server Performance

Let's benchmark the server's throughput and latency under concurrent load.


In [ ]:
# Benchmark the server
import concurrent.futures
import time
import statistics

def send_request(prompt, max_tokens=256):
    """Send a single request and measure latency"""
    start_time = time.time()
    
    response = client.completions.create(
        model="meta-llama/Llama-3.1-8B-Instruct",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0.7
    )
    
    latency = time.time() - start_time
    tokens = response.usage.total_tokens
    
    return {
        'latency': latency,
        'tokens': tokens,
        'tokens_per_second': tokens / latency
    }

# Test prompts
test_prompts = [
    "Write a short story about a robot learning to paint.",
    "Explain quantum computing to a 5-year-old.",
    "What are the key features of Python programming?",
    "Describe the process of photosynthesis.",
    "What is the history of artificial intelligence?",
]

print("Running benchmark with 5 concurrent requests...")
print("="*60)

start_time = time.time()

# Send concurrent requests
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(send_request, prompt) for prompt in test_prompts]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

total_time = time.time() - start_time

# Calculate statistics
latencies = [r['latency'] for r in results]
tokens_per_sec = [r['tokens_per_second'] for r in results]
total_tokens = sum(r['tokens'] for r in results)

print(f"\nBenchmark Results:")
print(f"  Total time: {total_time:.2f}s")
print(f"  Total tokens: {total_tokens}")
print(f"  Average latency: {statistics.mean(latencies):.2f}s")
print(f"  P50 latency: {statistics.median(latencies):.2f}s")
print(f"  P95 latency: {sorted(latencies)[int(len(latencies)*0.95)]:.2f}s")
print(f"  Average throughput: {statistics.mean(tokens_per_sec):.2f} tokens/s")
print(f"  Total throughput: {total_tokens/total_time:.2f} tokens/s")


## 8. Serving Multi-Modal Models (Vision + Language)

SGLang supports vision-language models like LLaVA. Let's try serving a multi-modal model.

**Note**: This requires more GPU memory. Skip this section if you have < 24GB VRAM.


In [ ]:
# Launch SGLang with LLaVA model (Vision-Language)
# Stop previous server first
try:
    requests.post("http://127.0.0.1:30000/shutdown", timeout=5)
    time.sleep(5)
    print("Previous server stopped.")
except:
    print("No previous server to stop.")

def run_multimodal_server():
    """Run SGLang server with LLaVA (vision-language model)"""
    global server_process
    
    cmd = [
        "python3", "-m", "sglang.launch_server",
        "--model-path", "lmms-lab/llama3-llava-next-8b",
        "--port", "30000",
        "--host", "0.0.0.0",
        "--tp-size", "1",  # Tensor parallelism size
        "--chat-template", "llava_llama_3"
    ]
    
    server_process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    for line in server_process.stdout:
        print(line, end='')
    
    return server_process

print("Launching SGLang server with LLaVA multi-modal model...")
print("This may take longer as it's a larger model...")
print("="*60)

multimodal_thread = threading.Thread(target=run_multimodal_server, daemon=True)
multimodal_thread.start()

time.sleep(45)
print("\nMulti-modal server ready! You can now query it with images.")


## 9. Using the HTTP API Directly

SGLang exposes an OpenAI-compatible HTTP API. You can query it with curl or any HTTP client.


In [ ]:
# Query the server using HTTP API directly
import requests
import json

# Health check
_ = health = requests.get("http://127.0.0.1:30000/health")
print(f"Server health: {health.json()}")

# Get model info
_ = model_info = requests.get("http://127.0.0.1:30000/v1/models")
print(f"\nAvailable models: {json.dumps(model_info.json(), indent=2)}")

# Send a completion request via HTTP
payload = {
    "model": "meta-llama/Llama-3.1-8B-Instruct",
    "prompt": "What are the benefits of using SGLang?",
    "max_tokens": 200,
    "temperature": 0.7,
    "stream": False
}

_ = response = requests.post(
    "http://127.0.0.1:30000/v1/completions",
    json=payload,
    headers={"Content-Type": "application/json"}
)

result = response.json()
print(f"\nHTTP API Response:")
print(f"Prompt: {payload['prompt']}")
print(f"Completion: {result['choices'][0]['text']}")
print(f"Tokens used: {result['usage']['total_tokens']}")


## 10. Best Practices & Tips

### Memory Optimization
- Use `--mem-fraction-static` to control KV cache size (default 0.9)
- Enable quantization: `--quantization fp8` or `--quantization awq`
- Use smaller context lengths for higher throughput

### Performance Tuning
- Adjust batch size with `--max-running-requests` (default 4096)
- Use tensor parallelism for large models: `--tp-size 2` or `--tp-size 4`
- Enable continuous batching for better throughput

### Common Issues
1. **CUDA Out of Memory**: Reduce `--mem-fraction-static` or use quantization
2. **Slow First Request**: Model loading takes time; subsequent requests are fast
3. **Port Already in Use**: Change port with `--port 30001` or kill existing process

### Production Deployment
- Use Docker for reproducible deployments
- Set up monitoring with Prometheus/Grafana
- Use a load balancer for multiple instances
- Enable logging: `--log-level info`

### Useful Commands
```bash
# Check SGLang version
python -m sglang.version

# List available models
python -m sglang.list_models

# Run benchmarks
python -m sglang.bench_serving \
  --backend sglang \
  --model meta-llama/Llama-3.1-8B-Instruct \
  --port 30000 \
  --dataset-name random \
  --random-input 512 \
  --random-output 256 \
  --max-concurrency 16
```

### Resources
- [SGLang GitHub](https://github.com/sgl-project/sglang)
- [SGLang Documentation](https://docs.sglang.ai/)
- [FlashInfer Kernels](https://github.com/flashinfer-ai/flashinfer)
- [Model Compatibility List](https://docs.sglang.ai/backend/supported_models.html)


## 11. Advanced: Streaming Responses

SGLang supports streaming responses for real-time output.


In [ ]:
# Example of streaming responses
print("Testing streaming response...")
print("="*60)

stream = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": "Write a haiku about GPU computing."}
    ],
    max_tokens=100,
    temperature=0.8,
    stream=True
)

print("\nStreaming response:")
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='', flush=True)

print("\n\n" + "="*60)
print("Streaming complete!")


## 🔧 Debugging Tools

If something isn't working, use this cell to diagnose issues:


In [ ]:
import subprocess
import requests

print("=" * 70)
print("🔧 SYSTEM DIAGNOSTICS")
print("=" * 70)

# 1. GPU Status
print("\n[1] GPU Memory Status:")
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,name,memory.used,memory.total', '--format=csv,noheader'],
        capture_output=True,
        text=True,
        timeout=5
    )
    print(result.stdout)
except Exception as e:
    print(f"✗ Could not get GPU status: {e}")

# 2. Running SGLang Processes
print("\n[2] SGLang Processes:")
try:
    result = subprocess.run(
        ["ps", "aux"],
        capture_output=True,
        text=True,
        timeout=5
    )
    sglang_procs = [line for line in result.stdout.split('\n') if 'sglang.launch_server' in line]
    if sglang_procs:
        print("  Found {} SGLang process(es):".format(len(sglang_procs)))
        for proc in sglang_procs:
            print(f"  {proc}")
    else:
        print("  ✗ No SGLang processes found")
except Exception as e:
    print(f"✗ Could not list processes: {e}")

# 3. Port Status
print("\n[3] Port 30000 Status:")
try:
    result = subprocess.run(
        ["lsof", "-i", ":30000"],
        capture_output=True,
        text=True,
        timeout=5
    )
    if result.stdout:
        print(f"  Port 30000 is IN USE:")
        print(result.stdout)
    else:
        print("  Port 30000 is FREE")
except Exception as e:
    print(f"  Port 30000 appears free (or lsof not available)")

# 4. Server Health Check
print("\n[4] Server Health Check:")
try:
    response = requests.get("http://127.0.0.1:30000/health", timeout=3)
    print(f"  ✓ Server is responding: {response.json()}")
except requests.exceptions.ConnectionError:
    print("  ✗ Cannot connect to server on port 30000")
except Exception as e:
    print(f"  ✗ Health check failed: {e}")

# 5. Server Logs (last 20 lines)
print("\n[5] Server Logs (last 20 lines):")
try:
    result = subprocess.run(
        ["tail", "-20", "/tmp/sglang_server.log"],
        capture_output=True,
        text=True,
        timeout=5
    )
    if result.stdout:
        print(result.stdout)
    else:
        print("  No logs found at /tmp/sglang_server.log")
except Exception as e:
    print(f"  Could not read logs: {e}")

print("\n" + "=" * 70)
print("✓ Diagnostics complete")
print("=" * 70)


## 12. Cleanup

Let's stop the server and clean up resources.


In [ ]:
import subprocess
import time

print("=" * 70)
print("🧹 CLEANUP - STOPPING ALL SGLANG SERVERS")
print("=" * 70)

# Step 1: Try graceful shutdown via API
print("\n[1/3] Attempting graceful shutdown...")
try:
    import requests
    response = requests.post("http://127.0.0.1:30000/shutdown", timeout=5)
    if response.status_code == 200:
        print("✓ Server acknowledged shutdown request")
        time.sleep(3)
    else:
        print(f"⚠️  Server returned status {response.status_code}")
except Exception as e:
    print(f"⚠️  Could not reach server API: {e}")

# Step 2: Kill all sglang processes
print("\n[2/3] Stopping all SGLang processes...")
try:
    result = subprocess.run(
        ["pkill", "-9", "-f", "sglang.launch_server"],
        capture_output=True,
        timeout=5
    )
    time.sleep(2)
    
    # Check if any processes remain
    check = subprocess.run(
        ["pgrep", "-f", "sglang.launch_server"],
        capture_output=True
    )
    if check.returncode == 0:
        print("⚠️  Some processes may still be running")
        print("  Try manually: !pkill -9 -f sglang.launch_server")
    else:
        print("✓ All SGLang processes stopped")
except Exception as e:
    print(f"⚠️  Could not kill processes: {e}")

# Step 3: Clean up temp files
print("\n[3/3] Cleaning up temporary files...")
try:
    import os
    if os.path.exists('/tmp/sglang_server.log'):
        log_size = os.path.getsize('/tmp/sglang_server.log')
        print(f"✓ Server logs available: /tmp/sglang_server.log ({log_size} bytes)")
        print("  Run to view: !tail -50 /tmp/sglang_server.log")
    else:
        print("  No log files found")
except Exception as e:
    print(f"⚠️  Could not check temp files: {e}")

print("\n" + "=" * 70)
print("✅ CLEANUP COMPLETE")
print("=" * 70)

print("\n🎉 Tutorial complete! You now know how to:")
print("  ✓ Install and configure SGLang on NVIDIA GPUs")
print("  ✓ Launch servers with proper process management")
print("  ✓ Query servers using Python and HTTP APIs")
print("  ✓ Leverage RadixAttention for massive speedups")
print("  ✓ Benchmark and optimize for production")
print("  ✓ Debug issues with diagnostic tools")

print("\n📚 Next steps:")
print("  - Try different models from HuggingFace")
print("  - Experiment with FP8 quantization")
print("  - Test multi-modal models (LLaVA)")
print("  - Deploy to production with Docker")
print("  - Scale with tensor parallelism on multi-GPU setups")

print("\n💡 Resources:")
print("  - SGLang Docs: https://docs.sglang.ai/")
print("  - GitHub: https://github.com/sgl-project/sglang")
print("  - Discord: https://discord.gg/NVDyv7TUgJ")


---

## Need Help?

Join the community:
- [SGLang GitHub Discussions](https://github.com/sgl-project/sglang/discussions)
- [NVIDIA Brev Discord](https://discord.gg/NVDyv7TUgJ)
- [SGLang Documentation](https://docs.sglang.ai/)

Built with 💚 by the NVIDIA Brev team
